In [4]:
import os
import torch
import numpy as np
from model import UCCDRNModel

experiment_id = "189454739472380536"
run_id = "98d0cca708cc4f5ba40314aa134af4cd"

model = torch.load(os.path.join("mlruns", experiment_id, run_id, "artifacts/best_model/data/model.pth"), weights_only=False)

In [5]:
data_dir = "../data/mnist/splitted_mnist_dataset.npz"
splitted_dataset = np.load(data_dir)

x_train = splitted_dataset["x_train"]
x_train = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1) / 255
x_train = (x_train - x_train.mean(dim=(2, 3), keepdim=True)) / x_train.std(
    dim=(2, 3), keepdim=True
)

# elif self.mode =="val":
x_val = splitted_dataset["x_val"]
x_val = torch.tensor(x_val, dtype=torch.float32).unsqueeze(1) / 255
x_val = (x_val - x_val.mean(dim=(2, 3), keepdim=True)) / x_val.std(
    dim=(2, 3), keepdim=True
)
x_test = splitted_dataset["x_test"]
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1) / 255
x_test = (x_test - x_test.mean(dim=(2, 3), keepdim=True)) / x_test.std(
    dim=(2, 3), keepdim=True
)

In [6]:
device = torch.device("mps")
x_train_encoded = []
x_val_encoded = []
x_test_encoded = []
with torch.no_grad():
    model.eval()
    batches = int(len(x_train)/200)
    for i in range(batches):
        tensor = x_train[i*200: (i+1)*200]
        tensor = tensor.to(torch.float32).to(device)
        outputs = model.encoder(tensor)
        outputs = outputs.cpu().numpy()
        if len(x_train_encoded)==0:
            x_train_encoded = outputs
        else:
            x_train_encoded = np.concat((x_train_encoded, outputs))
with torch.no_grad():
    model.eval()
    batches = int(len(x_val)/200)
    for i in range(batches):
        tensor = x_val[i*200: (i+1)*200]
        tensor = tensor.to(torch.float32).to(device)
        outputs = model.encoder(tensor)
        outputs = outputs.cpu().numpy()
        if len(x_val_encoded)==0:
            x_val_encoded = outputs
        else:
            x_val_encoded = np.concat((x_val_encoded, outputs))
with torch.no_grad():
    model.eval()
    batches = int(len(x_test)/200)
    for i in range(batches):
        tensor = x_test[i*200: (i+1)*200]
        tensor = tensor.to(torch.float32).to(device)
        outputs = model.encoder(tensor)
        outputs = outputs.cpu().numpy()
        if len(x_test_encoded)==0:
            x_test_encoded = outputs
        else:
            x_test_encoded = np.concat((x_test_encoded, outputs))

In [7]:
splitted_dataset = dict(splitted_dataset)
splitted_dataset["x_train"] = x_train_encoded
splitted_dataset["x_val"] = x_val_encoded
splitted_dataset["x_test"] = x_test_encoded

np.savez("../data/mnist/splitted_drn_mnist_encoded_dataset.npz", **splitted_dataset)

In [8]:
from dataset import DRNMnistEncodedDataset
from hydra import compose, initialize

with initialize(version_base=None, config_path="../configs"):
        cfg = compose(config_name="train_drn")


In [9]:
args = cfg.args
train_dataset_len = args.train_num_steps * args.batch_size
dataset = DRNMnistEncodedDataset(
    mode="train",
    num_instances=args.num_instances,
    num_samples_per_class=args.num_samples_per_class,
    digit_arr=list(range(args.ucc_end-args.ucc_start+1)),
    ucc_start=args.ucc_start,
    ucc_end=args.ucc_end,
    length=train_dataset_len,
)

x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples
dict_keys(['x_val', 'y_train', 'y_val', 'x_test', 'y_test', 'x_train'])
x_train shape: torch.Size([50000, 10])
50000 train samples
10000 val samples


In [40]:
import torch
model = torch.nn.Linear(10,10)
optimizer = torch.optim.Adam(model.parameters(), 0.01)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 100, 1.2, eta_min=0.00005
)

ValueError: Expected integer T_mult >= 1, but got 1.2

In [41]:
for epoch in range(1000, 10000, 10):
    lr_scheduler.step(epoch)
    print(lr_scheduler.get_last_lr())

[0.01]
[0.00975650616856839]
[0.009049859547015364]
[0.007949231630155056]
[0.006562359547015364]
[0.005025]
[0.0034876404529846383]
[0.002100768369844947]
[0.001000140452984637]
[0.0002934938314316112]
[0.01]
[0.00975650616856839]
[0.009049859547015364]
[0.007949231630155056]
[0.006562359547015364]
[0.005025]
[0.0034876404529846383]
[0.002100768369844947]
[0.001000140452984637]
[0.0002934938314316112]
[0.01]
[0.00975650616856839]
[0.009049859547015364]
[0.007949231630155056]
[0.006562359547015364]
[0.005025]
[0.0034876404529846383]
[0.002100768369844947]
[0.001000140452984637]
[0.0002934938314316112]
[0.01]
[0.00975650616856839]
[0.009049859547015364]
[0.007949231630155056]
[0.006562359547015364]
[0.005025]
[0.0034876404529846383]
[0.002100768369844947]
[0.001000140452984637]
[0.0002934938314316112]
[0.01]
[0.00975650616856839]
[0.009049859547015364]
[0.007949231630155056]
[0.006562359547015364]
[0.005025]
[0.0034876404529846383]
[0.002100768369844947]
[0.001000140452984637]
[0.000293

In [43]:
def load_model_and_optimizer(experiment_id, run_id):
    model = torch.load(f"mlruns/{experiment_id}/{run_id}/artifacts/best_model/data/model.pth", weights_only=False)
    # old_optimizer = torch.load(f"mlruns/{experiment_id}/{run_id}/artifacts/best_optimizer.pth/best_optimizer.pth", weights_only=False)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # optimizer.load_state_dict(old_optimizer.state_dict())
    with open(f"mlruns/{experiment_id}/{run_id}/metrics/eval_ucc_acc") as file:
        lines = file.readlines()
        step = int(lines[-1].split(" ")[-1])
    return model, optimizer, step
experiment_id = "189454739472380536"
run_id = "98d0cca708cc4f5ba40314aa134af4cd"
model, optimizer, step = load_model_and_optimizer(experiment_id=experiment_id, run_id=run_id)

In [ ]:
drn= model.ucc_classifier


input_layer = drn[0]

In [71]:
weights = []
ba = []
bq = []
lama = []
lamq = []
for layer in drn[:-1]:
    weights.append(layer.W)
    ba.append(layer.ba)
    bq.append(layer.bq)
    lama.append(layer.lama)
    lamq.append(layer.lamq)

In [76]:
from collections import defaultdict
statistics = {}

statistics["w"] = []
for index, w in enumerate(weights):
    print(w.shape)
    stat_dict = {}
    stat_dict["max"] = w.max(dim=1)
    stat_dict["min"] = w.min(dim=1)
    stat_dict["std"] = w.std(dim=1)
    stat_dict["mean"] = w.mean(dim=1)
    statistics["w"].append(
        stat_dict
    )

torch.Size([9, 10])
torch.Size([9, 9])
torch.Size([9, 9])
torch.Size([1, 9])


In [77]:
statistics

{'w': [{'max': torch.return_types.max(
   values=tensor([4.1258, 5.4059, 2.6690, 4.6601, 2.5000, 2.7482, 3.9410, 3.4813, 5.1677],
          device='mps:0', grad_fn=<MaxBackward0>),
   indices=tensor([1, 6, 2, 7, 6, 6, 7, 6, 4], device='mps:0')),
   'min': torch.return_types.min(
   values=tensor([-6.5798, -6.8592,  0.1061,  0.0466, -3.2616, -1.2818, -1.8255,  0.2992,
           -6.4203], device='mps:0', grad_fn=<MinBackward0>),
   indices=tensor([0, 8, 4, 4, 4, 4, 1, 7, 1], device='mps:0')),
   'std': tensor([3.9890, 3.7651, 0.8355, 1.1779, 1.7089, 1.3231, 1.9351, 1.1359, 4.3443],
          device='mps:0', grad_fn=<StdBackward0>),
   'mean': tensor([-1.9596, -0.5349,  1.0705,  1.6854,  1.0741,  1.2019,  1.5159,  1.5380,
           -2.6157], device='mps:0', grad_fn=<MeanBackward1>)},
  {'max': torch.return_types.max(
   values=tensor([ 6.4013,  6.1084, -3.0711, -3.0662, 12.4177,  5.1276, 12.6260, -3.0465,
           -2.9798], device='mps:0', grad_fn=<MaxBackward0>),
   indices=tensor([7

In [81]:
import numpy as np

np.power(np.e, -27)

np.float64(1.879528816539086e-12)

In [85]:
0.000000000001

1e-12

In [59]:
input_layer.lama

Parameter containing:
tensor([[ 1.0219],
        [ 1.0226],
        [-0.0360],
        [ 0.4180],
        [ 0.5661],
        [-0.0155],
        [ 0.4543],
        [ 0.5150],
        [ 0.8170]], device='mps:0', requires_grad=True)

In [56]:
layer_1 = drn[1]

In [63]:
layer_1.bq

Parameter containing:
tensor([[-1.7307],
        [ 0.6836],
        [-0.7373],
        [-0.6748],
        [-1.3735],
        [ 0.7535],
        [-1.4816],
        [-1.0582],
        [-0.3545]], device='mps:0', requires_grad=True)